In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader 
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

### Using Cross-entropy Label Smooth as the loss function as it works well than cross-entropy traditionally.

In [2]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import math

class CrossEntropyLabelSmooth(nn.Module):
    """Cross entropy loss with label smoothing regularizer.
    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.
    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """
    def __init__(self, num_classes, epsilon=0.1, device='cpu'):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.device = device
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        log_probs = self.logsoftmax(inputs)
        # targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data, 1)# for mldg da
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)#for zzd
        targets = targets.to(self.device)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (-Variable(targets) * log_probs).mean(0).sum()
        return loss

### Own implemented dataset importer.

In [3]:
class FoodDataset(Dataset):
    def __init__(self, file, transform=None, mode='train'):
        self.transforms = transform
        self.mode = mode
        with open(file, 'r') as f:
            self.image_list = f.readlines()

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        label = None
        if self.mode == 'train':
            image, label = self.image_list[index].split('\n')[0].split('\t')
            label = int(label)
        else:
            image = self.image_list[index].split('\n')[0]
        image = Image.open(image).convert('RGB')
        image = self.transforms(image)
        if self.mode == 'train':
            return image, label
        else:
            return image

# Augmentation
Augmentations like random flip, random crop is necessary as it brings much improvement to the model.

In [4]:
transforms_train = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.Pad(10, 10),
                transforms.RandomRotation(45),
                transforms.RandomCrop((224, 224)),
                transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

transforms_test = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

### Function for accuracy validation

In [5]:
def evaluate(prediction, ground_truth):
    num_correct = (np.array(prediction) == np.array(ground_truth)).sum()
    return num_correct / len(prediction)

In [6]:
train_ds = FoodDataset('/media/ntu/volume2/home/s121md302_07/food/data/train.txt', transform=transforms_train)
val_ds = FoodDataset('/media/ntu/volume2/home/s121md302_07/food/data/val.txt', transform=transforms_test)
test_ds = FoodDataset('/media/ntu/volume2/home/s121md302_07/food/data/test.txt', transform=transforms_test)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=8)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=True, num_workers=8)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=True, num_workers=8)

## Hyperparameters
Using ResNet34 with Adam optimizer and StrpLR scheduler in this experiment.
The learning rate is set initially to 0.0001 and decay by 10 times every 20 epochs.

In [7]:
num_classes = 5
train_model = models.resnet34(pretrained=True)
train_model.fc = nn.Linear(512, num_classes)

model_str = 'resnet34'
output_dir = 'checkpoint_' + model_str
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ce_loss = CrossEntropyLabelSmooth(num_classes = num_classes, device = device)
optimizer = torch.optim.Adam(train_model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

## Training with model freezed
Freezing the model will help the model to converge faster in the future.

In [8]:
for param in train_model.parameters():
    param.requires_grad = False
for param in train_model.fc.parameters():
    param.requires_grad = True
for i in range(5):
    train_model.train()
    train_model.to(device)
    for img, label in tqdm(train_dl):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output= train_model(img) 
        loss = ce_loss(output, label) 
        loss.backward()
        optimizer.step()


100%|██████████| 16/16 [00:01<00:00, 13.38it/s]


# Training
Training for 100 epochs. Validation and checkpoint at every 10 epochs.

In [9]:
for param in train_model.parameters():
    param.requires_grad = True
epoch = 100
highest_acc = {'epoch': 0, 'accuracy': 0}
for ep in range(epoch):
    train_model.train()
    train_model.to(device)
    count = 0
    running_loss = 0.0
    validation_loss = 0.0
    output_list = []
    ground_truth_list = []
    for img, label in tqdm(train_dl):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output= train_model(img)
        loss = ce_loss(output, label)
        count += 1
        prediction = torch.argmax(output, dim=1)
        output_list.extend(prediction.detach().cpu())
        ground_truth_list.extend(label.cpu())
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    scheduler.step()
        
    if ep % 10 == 0:
        torch.save(train_model.state_dict(), output_dir + '/' + model_str + '_' + str(ep) + '.pth')
        
    accuracy = evaluate(output_list, ground_truth_list)
    print(f'Epoch[{ep}] training accuracy: {accuracy} '
            f'training loss: {running_loss / count:.3e} Base Lr: {optimizer.param_groups[0]["lr"]:.5e}')

    if ep % 10 == 0:
        train_model.eval()
        count = 0
        output_list = []
        ground_truth_list = []
        for img, label in tqdm(val_dl):
            with torch.no_grad():
                img = img.to(device)
                lbl = label.to(device)

                output= train_model(img)

                val_loss = ce_loss(output, lbl)

                validation_loss += val_loss.item()
                count += 1
                prediction = torch.argmax(output, dim=1)
                output_list.extend(prediction.detach().cpu())
                ground_truth_list.extend(label)
        accuracy = evaluate(output_list, ground_truth_list)
        if accuracy > highest_acc['accuracy']:
            highest_acc['accuracy'] = accuracy
            highest_acc['epoch'] = ep
        print(f'Accuracy: {accuracy}    Epoch:{ep}')

100%|██████████| 16/16 [00:01<00:00,  9.45it/s]


Epoch[0] training accuracy: 0.592 training loss: 1.209e+00 Base Lr: 1.00000e-04


100%|██████████| 5/5 [00:00<00:00,  5.71it/s]


Accuracy: 0.76    Epoch:0


100%|██████████| 16/16 [00:01<00:00,  9.72it/s]


...

Epoch[97] training accuracy: 0.998 training loss: 4.173e-01 Base Lr: 1.00000e-08


100%|██████████| 16/16 [00:01<00:00,  9.96it/s]


Epoch[98] training accuracy: 1.0 training loss: 4.188e-01 Base Lr: 1.00000e-08


100%|██████████| 16/16 [00:01<00:00,  9.70it/s]

Epoch[99] training accuracy: 1.0 training loss: 4.117e-01 Base Lr: 1.00000e-09


### Show the highest accuracy and save the final weight of the model.

In [10]:
torch.save(train_model.state_dict(), output_dir + '/' + model_str + '_' + 'final' + '.pth')
print('highest_acc: {}  epoch: {}'.format(highest_acc['accuracy'], highest_acc['epoch']))

highest_acc: 0.9266666666666666  epoch: 50
